In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os
import numpy as np
from PIL import Image

In [20]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Data transforms
transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),
])

# Load the datasets
train_data = datasets.ImageFolder(root=r'D:\junotbok\9th week\dog vs cat\dog vs cat\dataset\training_set', transform=transform)
validation_data = datasets.ImageFolder(root=r'D:\junotbok\9th week\dog vs cat\dog vs cat\dataset\test_set', transform=transform)

# Data loaders
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_data, batch_size=32, shuffle=False)


In [21]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 18 * 18, 512)
        self.fc2 = nn.Linear(512, 1)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 18 * 18)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x


In [22]:
import torch.optim as optim

# Initialize the model, criterion, and optimizer
model = CNN()
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:

# Training loop
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        labels = labels.float().unsqueeze(1)  # Reshape labels for binary classification
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
         
        running_loss += loss.item()
    
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}')


Epoch 1, Loss: 0.6819665298461914
Epoch 2, Loss: 0.6247330054044723
Epoch 3, Loss: 0.5779979345798493
Epoch 4, Loss: 0.5301806920766831
Epoch 5, Loss: 0.4839596242904663


In [ ]:
# Save the model
torch.save(model.state_dict(), 'cats_and_dogs_cnn.pth')

# Load the model
model = CNN()
model.load_state_dict(torch.load('cats_and_dogs_cnn.pth'))
model.eval()

In [ ]:
# Example for using the model on a new image
from PIL import Image

# Load and preprocess the new image
img = Image.open('new_image.jpg')
img = transform(img).unsqueeze(0)  # Transform and add batch dimension

# Predict
with torch.no_grad():
    output = model(img)
    prediction = output.round().item()

if prediction == 1:
    print("It's a dog!")
else:
    print("It's a cat!")